In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mh_file_parser import parse
import polars as pl
import plotly.express as px

In [3]:

inputfile = open("./sampledata/example-10mins-ch1-t2.ptu", "rb")
result = parse(inputfile)

Writing 3344031 records, this may take a while...
{'globRes': 5e-12, 'numRecords': 3344031}
Progress: 98.7%

In [4]:
# show channel: event count
[f"{i}: {len(ch)}" for i,ch in enumerate(result.events) if len(ch) > 0]

['0: 2675363']

In [5]:
# generate random data
import random

data = result.events[0]
df = pl.DataFrame(
    {
        "timestamp": data,
        "sync": [1] * len(data),
        "ch1": [1] * len(data),
        "ch2": [int(random.random() * 2) for _ in range(0, len(data))],
        "ch3": [int(random.random() * 2) for _ in range(0, len(data))],
        "ch4": [int(random.random() * 2) for _ in range(0, len(data))],
    }
)
print(df)

shape: (2_675_363, 6)
┌─────────────────┬──────┬─────┬─────┬─────┬─────┐
│ timestamp       ┆ sync ┆ ch1 ┆ ch2 ┆ ch3 ┆ ch4 │
│ ---             ┆ ---  ┆ --- ┆ --- ┆ --- ┆ --- │
│ i64             ┆ i64  ┆ i64 ┆ i64 ┆ i64 ┆ i64 │
╞═════════════════╪══════╪═════╪═════╪═════╪═════╡
│ 493506484       ┆ 1    ┆ 1   ┆ 1   ┆ 1   ┆ 0   │
│ 493550903       ┆ 1    ┆ 1   ┆ 0   ┆ 1   ┆ 1   │
│ 493554764       ┆ 1    ┆ 1   ┆ 0   ┆ 1   ┆ 0   │
│ 493593436       ┆ 1    ┆ 1   ┆ 0   ┆ 1   ┆ 1   │
│ …               ┆ …    ┆ …   ┆ …   ┆ …   ┆ …   │
│ 119999607661897 ┆ 1    ┆ 1   ┆ 1   ┆ 1   ┆ 1   │
│ 119999608902507 ┆ 1    ┆ 1   ┆ 0   ┆ 0   ┆ 1   │
│ 119999608948135 ┆ 1    ┆ 1   ┆ 1   ┆ 1   ┆ 0   │
│ 119999610840642 ┆ 1    ┆ 1   ┆ 0   ┆ 0   ┆ 1   │
└─────────────────┴──────┴─────┴─────┴─────┴─────┘


In [6]:
# resample data by given interval, and aggregate data by channels
interval = 100000
resampled_df = (
    df.with_columns(pl.col("timestamp") / interval)
    .with_columns(pl.col("timestamp").cast(pl.Int64))
    .group_by("timestamp")
    .agg(
        pl.col("sync").sum().alias("sync"),
        pl.col("ch1").sum().alias("ch1"),
        pl.col("ch2").sum().alias("ch2"),
        pl.col("ch3").sum().alias("ch3"),
        pl.col("ch4").sum().alias("ch4"),
        (pl.col("sync") + pl.col("ch1")).sum().alias("sync&1"),
        (pl.col("sync") + pl.col("ch2")).sum().alias("sync&2"),
        (pl.col("sync") + pl.col("ch1") + pl.col("ch2")).sum().alias("sync&1&2"),
        (pl.col("ch1") + pl.col("ch2")).sum().alias("1&2"),
        (pl.col("ch1") + pl.col("ch3")).sum().alias("1&3"),
        (pl.col("ch2") + pl.col("ch4")).sum().alias("2&4"),
        (pl.col("ch3") + pl.col("ch4")).sum().alias("3&4"),
        (pl.col("ch1") + pl.col("ch2") + pl.col("ch4")).sum().alias("1&2&4"),
        (pl.col("ch1") + pl.col("ch2") + pl.col("ch3") + pl.col("ch4"))
        .sum()
        .alias("1&2&3&4"),
        (pl.col("sync") + pl.col("ch1") + pl.col("ch2") + pl.col("ch3") + pl.col("ch4"))
        .sum()
        .alias("sum"),
    )
    .sort("timestamp")
    .with_columns(pl.col("timestamp") * interval)
)
print(resampled_df)

shape: (1_429_990, 16)
┌─────────────────┬──────┬─────┬─────┬───┬─────┬───────┬─────────┬─────┐
│ timestamp       ┆ sync ┆ ch1 ┆ ch2 ┆ … ┆ 3&4 ┆ 1&2&4 ┆ 1&2&3&4 ┆ sum │
│ ---             ┆ ---  ┆ --- ┆ --- ┆   ┆ --- ┆ ---   ┆ ---     ┆ --- │
│ i64             ┆ i64  ┆ i64 ┆ i64 ┆   ┆ i64 ┆ i64   ┆ i64     ┆ i64 │
╞═════════════════╪══════╪═════╪═════╪═══╪═════╪═══════╪═════════╪═════╡
│ 493500000       ┆ 4    ┆ 4   ┆ 1   ┆ … ┆ 6   ┆ 7     ┆ 11      ┆ 15  │
│ 744000000       ┆ 5    ┆ 5   ┆ 2   ┆ … ┆ 5   ┆ 11    ┆ 12      ┆ 17  │
│ 1093400000      ┆ 2    ┆ 2   ┆ 2   ┆ … ┆ 2   ┆ 4     ┆ 6       ┆ 8   │
│ 1193300000      ┆ 1    ┆ 1   ┆ 1   ┆ … ┆ 1   ┆ 2     ┆ 3       ┆ 4   │
│ …               ┆ …    ┆ …   ┆ …   ┆ … ┆ …   ┆ …     ┆ …       ┆ …   │
│ 119999607500000 ┆ 2    ┆ 2   ┆ 0   ┆ … ┆ 1   ┆ 2     ┆ 3       ┆ 5   │
│ 119999607600000 ┆ 3    ┆ 3   ┆ 2   ┆ … ┆ 3   ┆ 6     ┆ 8       ┆ 11  │
│ 119999608900000 ┆ 2    ┆ 2   ┆ 1   ┆ … ┆ 2   ┆ 4     ┆ 5       ┆ 7   │
│ 119999610800000 ┆ 1    ┆ 1

In [ ]:
# show raw data histogram
fig = px.histogram(df, x="timestamp", nbins=int(interval))
fig.update_layout(bargap=0.2)


In [8]:
# fig = px.bar(resampled_df, x="timestamp", y="ch1")
# fig.show()